In [1]:
# taken partially from tutorial at: https://stmorse.github.io/journal/espn-fantasy-python.html
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import requests

league1 = 380134
league2 = 1553391
leagues=[league1, league2]

weeks = 5

scores={}

for i in range(len(leagues)):
    scores[i] = {}
for week in range(1, weeks+1):
    for league in range(len(leagues)):
        r = requests.get('http://games.espn.com/ffl/api/v2/scoreboard', 
                         params={'leagueId': leagues[league], 'seasonId': 2018, 'matchupPeriodId': week})
        scores[league][week] = r.json()

df = []
dfRecord = []
for league in range(len(leagues)):
    for week in scores[league]:
        temp = scores[league][week]['scoreboard']['matchups']
        for match in temp:
            
            df.append([week, 
                       match['teams'][0]['team']['teamAbbrev'],
                       match['teams'][1]['team']['teamAbbrev'],
                       match['teams'][0]['score'],
                       match['teams'][1]['score']
                      ])


df = pd.DataFrame(df, columns=['Week', 'HomeAbbrev', 'AwayAbbrev', 'HomeScore', 'AwayScore'])
df.head()
df = (df[['Week', 'HomeAbbrev', 'HomeScore']]
      .rename(columns={'HomeAbbrev': 'Abbrev', 'HomeScore': 'Score'})
      .append(df[['Week', 'AwayAbbrev', 'AwayScore']]
             .rename(columns={'AwayAbbrev': 'Abbrev', 'AwayScore': 'Score'}))
     )

# now adding the record:

for league in range(len(leagues)):
    temp2=scores[league][weeks]['scoreboard']['matchups']
    for match in temp2:
        for team in match['teams']:
#         append "points for" for debugging purposes
            dfRecord.append([
                            team['team']['teamAbbrev'],
                            team['team']['record']['pointsFor'],
                            team['team']['record']['pointsAgainst'],
                            team['team']['record']['overallWins'],
                            team['team']['record']['overallLosses'],
                            team['team']['record']['overallTies']
                            ])
dfRecord = pd.DataFrame(dfRecord, columns=["Abbrev", "Points For", "Points Against", "Overall Wins","Overall Losses", "Overall Ties"])
dfRecord = dfRecord.set_index('Abbrev')
df1 = df.groupby("Abbrev")["Score"].describe().drop("count", axis=1)
s1 = df.groupby("Abbrev")['Score'].sum()
finalDF = df1.join(dfRecord)
finalDF = finalDF.sort_values(by=['Overall Wins','Points For'], ascending=[False,False])
finalDF = finalDF.reset_index()
finalDF.index +=1
display(finalDF)

,Abbrev,mean,std,min,25%,50%,75%,max,Points For,Points Against,Overall Wins,Overall Losses,Overall Ties
1,LEE,105.776,15.696161,88.58,96.04,102.76,112.46,129.04,528.88,433.58,5,0,0
2,HARN,117.240,9.983877,105.20,109.14,120.64,121.34,129.88,586.20,529.58,4,1,0
3,LURI,113.404,15.536463,93.26,100.36,122.24,122.84,128.32,567.02,501.14,4,1,0
4,FITZ,108.788,7.355795,97.00,107.34,110.00,113.90,115.70,543.94,403.82,4,1,0
5,AUTO,107.572,21.629030,73.80,98.94,116.90,120.78,127.44,537.86,510.08,4,1,0
6,NATH,84.720,15.584595,67.90,75.34,77.74,100.46,102.16,423.60,383.20,4,1,0
7,PETR,116.724,46.046160,58.32,98.82,116.48,124.84,185.16,583.62,524.26,3,2,0
8,MACK,108.252,21.276995,86.06,95.04,99.42,123.76,136.98,541.26,497.42,3,2,0
9,AQP,107.228,7.984167,100.74,101.00,105.10,109.20,120.10,536.14,478.22,3,2,0
10,DREW,104.684,20.535279,78.48,90.00,106.30,122.00,126.64,523.42,461.08,3,2,0
